## NEON API DOWNLOAD & ANALYZE ASD DATA:
This notebook tests some functions for downloading data using the NEON API and running a basic analysis.

In [1]:
import requests, urllib, os

def list_available_urls(product,site):
    r = requests.get("http://data.neonscience.org/api/v0/products/" + product)
    for i in range(len(r.json()['data']['siteCodes'])):
        if site in r.json()['data']['siteCodes'][i]['siteCode']:
            data_urls=r.json()['data']['siteCodes'][i]['availableDataUrls']
    if len(data_urls)==0:
        print('WARNING: no urls found for product ' + product + ' at site ' + site)
    else:
        return data_urls

def list_available_urls_by_year(product,site,year):
    r = requests.get("http://data.neonscience.org/api/v0/products/" + product)
    for i in range(len(r.json()['data']['siteCodes'])):
        if site in r.json()['data']['siteCodes'][i]['siteCode']:
            all_data_urls=r.json()['data']['siteCodes'][i]['availableDataUrls']
    data_urls = [url for url in all_data_urls if year in url]
    if len(data_urls)==0:
        print('WARNING: no urls found for product ' + product + ' at site ' + site + ' in year ' + year)
    else:
        return data_urls
    
def download_urls(url_list,download_folder_root,zip=False):
    #downloads data from urls to folder, maintaining month-year folder structure
    for url in url_list:
        month = url.split('/')[-1]
        download_folder = download_folder_root + month + '/'
        if not os.path.exists(download_folder):
            os.makedirs(download_folder)
        r=requests.get(url)
        files=r.json()['data']['files']
        for i in range(len(files)):
            if zip==False:
                if '.zip' not in files[i]['name']:
                    print('downloading ' + files[i]['name'] + ' to ' + download_folder)
                    urllib.request.urlretrieve(files[i]['url'],download_folder + files[i]['name'])
            elif zip==True:
                if '.zip' in files[i]['name']:
                    print('downloading ' + files[i]['name'] + ' to ' + download_folder)
                    urllib.request.urlretrieve(files[i]['url'],download_folder + files[i]['name'])

In [2]:
def download_file(url,filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                return

def download_urls(data_urls,download_folder,ext=None): 
    #eg. download_folder = './data/lidar/...'
    #    ext = 'CHM.tif'
    for url in data_urls:
        r = requests.get(url)
        print(r.status_code)
        files = r.json()['data']['files']
        for i in range(len(files)):
            size=0
            if ext in files[i]['name']:
                print('downloading ' + files[i]['name'] + ' to ' + download_folder)
                size += int(files[i]['size'])
                try:
                    download_file(files[i]['url'],download_folder + files[i]['name'])
                    #urllib.request.urlretrieve(files[i]['url'], aop_lidar_data_folder
                except requests.exceptions.RequestException as e:
                    print(e)

In [ ]:
wref_fsp_urls = list_available_urls('DP1.30012.001','WREF')
wref_fsp_urls

In [ ]:
os.mkdir('./wref_fsp')
download_urls(wref_fsp_urls,'./wref_fsp/',zip=False)

In [ ]:
readme = open('./wref_fsp/2019-07/NEON.D16.WREF.DP1.30012.001.readme.20200210T191738Z.txt','r')
print(readme.read())
readme.close()

In [ ]:
import pandas as pd
variables = pd.read_csv('./wref_fsp/2019-07/NEON.D16.WREF.DP1.30012.001.variables.20200210T191738Z.csv')
variables.head(5)

In [ ]:
wref_chm_urls = list_available_urls_by_year('DP3.30015.001','WREF','2018')
wref_chm_urls

In [ ]:
# os.mkdir('./wref_chm')
download_urls(wref_chm_urls,'./wref_chm/','CHM.tif')

Product Name	Product Number
Spectrometer Orthrorectified at-Sensor Radiance	NEON.DOM.SITE.DP1.30008.001
LiDAR Slant Range Waveform	NEON.DOM.SITE.DP1.30001.001
Discrete Return LiDAR Point Cloud	NEON.DOM.SITE.DP1.30003.001
Spectrometer Orthorectified Surface Directional Reflectance	NEON.DOM.SITE.DP1.30006.001
High-resolution Orthorectified camera imagery	NEON.DOM.SITE.DP1.30010.001
Canopy Nitrogen	NEON.DOM.SITE.DP2.30018.001
Canopy Water Content	NEON.DOM.SITE.DP2.30019.001
Canopy Xanthophyll Cycle	NEON.DOM.SITE.DP2.30020.001
Canopy Lignin	NEON.DOM.SITE.DP2.30022.001
Vegetation Indices - Spectrometer	NEON.DOM.SITE.DP2.30026.001
Ecosystem Structure	NEON.DOM.SITE.DP3.30015.001
Elevation - LiDAR	NEON.DOM.SITE.DP3.30024.001
Slope and Aspect - LiDAR	NEON.DOM.SITE.DP3.30025.001
Albedo - Spectrometer	NEON.DOM.SITE.DP2.30011.001
LAI - Spectrometer	NEON.DOM.SITE.DP2.30012.001
fPAR - Spectrometer	NEON.DOM.SITE.DP2.30014.001
Total Biomass Map - Spectrometer	NEON.DOM.SITE.DP2.30016.001
Spectrometer Orthorectified Surface Directional Reflectance	NEON.DOM.SITE.DP3.30006.001
High-resolution Orthorectified camera imagery mosaic	NEON.DOM.SITE.DP3.30010.001
Albedo - Spectrometer	NEON.DOM.SITE.DP3.30011.001
LAI - Spectrometer	NEON.DOM.SITE.DP3.30012.001
fPAR - Spectrometer	NEON.DOM.SITE.DP3.30014.001
Total Biomass Map - Spectrometer	NEON.DOM.SITE.DP3.30016.001
Canopy Nitrogen	NEON.DOM.SITE.DP3.30018.001
Canopy Water Content	NEON.DOM.SITE.DP3.30019.001
Canopy Xanthophyll Cycle	NEON.DOM.SITE.DP3.30020.001
Canopy Lignin	NEON.DOM.SITE.DP3.30022.001
Vegetation Indices - Spectrometer	NEON.DOM.SITE.DP3.30026.001

In [ ]:
wref_vegind_urls = list_available_urls_by_year('DP3.30026.001','WREF','2018')

In [ ]:
wref_vegind_urls

In [ ]:
os.mkdir('./wref_nis')
download_urls(wref_vegind_urls,'./wref_nis/','580000_5075000_VegIndices.zip')

In [ ]:
wref_refl_urls = list_available_urls_by_year('DP3.30006.001','WREF','2018')
wref_refl_urls

In [ ]:
download_urls(wref_refl_urls,'./wref_nis/','580000_5075000_reflectance.h5')

Now let's download NEON_D17_SJER_DP3_257000_4112000_reflectance.h5 for the R test

In [3]:
sjer_h5_urls = list_available_urls_by_year('DP3.30006.001','SJER','2019')
# os.mkdir('./sjer_h5')
download_urls(sjer_h5_urls,'./sjer_h5','257000_4112000_reflectance.h5')

200
downloading NEON_D17_SJER_DP3_257000_4112000_reflectance.h5 to ./sjer_h5
